In [4]:
import numpy as np
import matplotlib as mpl
from matplotlib import figure, animation
import scipy as sci
mpl.rcParams.update({
    'font.family': 'serif',
    'text.usetex': True,
    'axes.labelsize': 'large',
    'figure.dpi': 100,
})

import os
import glob
import subprocess

In [72]:
def fig_io(fig, savefig, viewfig):
    '''
    Save figure and optionaly open it in an image viewer.
    '''
    if (not savefig) and viewfig:
        raise ValueError(
            'Setting `viewfig=True` is only'
            'valid when `savefig` is not `None`.'
        )
        
    if savefig:
        lowername = str.lower(savefig)
        is_img = (
            str.endswith(lowername, '.png')
            or str.endswith(lowername, '.jpg')
            or str.endswith(lowername, '.webp')
        )
        is_pdf = str.endswith(lowername, '.pdf')
        
        if not (is_img or is_pdf):
            raise ValueError(
                'Name for saved file `savefig` must have'
                'extension pdf, png, jpg or webp, '
                f'`{savefig}` is not a valid `savefig` value.'
            )
        
        # Save figure.
        absname = os.path.abspath(savefig)
        fig.savefig(absname, dpi=300)
            
        if viewfig:
            cmd = 'qview' if is_img else 'zathura'
            subprocess.Popen([cmd, absname])

In [73]:
# Import all measurements from the `./measurements` directory. Each measurement file
# is read into a numpy array (column-first) and added to the `meas_dict` (keys are
# filenames with extentions removed).
meas_dict = {}
suffix, sep = 'csv', ','
for fname in glob.glob(f'./measurements/*.{suffix}'):
    with open(fname) as file:
        meas = [[float(el) for el in row.rstrip().split(sep)]
                for row in file.readlines()]
        meas_name, _ = str.rsplit(os.path.basename(fname), '.', maxsplit=1)
        meas_dict[meas_name] = np.array(meas).T

In [207]:
ρ = 1.204 # Density of air
for key, val in meas_dict.items():
    U, I_nA = val
    I = 1e-9 * I_nA
    I = I[(100 <= U) & (U <= 300)]
    I_mean = np.mean(I)
    I_var = np.sum((I-I_mean)**2)
    ΔI_mean = np.sqrt(I_var/len(I))
    # Reference volume is 160 * (83+137)/2 * 34 [mm^3].
    V = 160e-3 * (83e-3+137e-3)/2 * 34e-3
    δV = (0.5/160 + 2*0.5/(83+137) + 0.5/34)
    doserate = I_mean / (ρ*V)
    Δdoserate = doserate * (ΔI_mean/I_mean + δV)
    print(f'{str.split(key, "-")[-1]}', end='')
    print(f' & ${1e9 * I_mean:.2f} \pm {1e9 * ΔI_mean:.2f}$', end='')
    print(f' & ${1e3 * 3600 * doserate:.2f} \pm {1e3 * 3600 * Δdoserate:.2f}$ \\\\', end='\n')

15 & $0.17 \pm 0.01$ & $0.83 \pm 0.07$ \\
25 & $1.20 \pm 0.02$ & $5.99 \pm 0.21$ \\
30 & $2.23 \pm 0.03$ & $11.12 \pm 0.42$ \\
35 & $3.52 \pm 0.04$ & $17.60 \pm 0.60$ \\


In [147]:
def fig_ionization(
    savefig=None,
    viewfig=False,
):
    fig = mpl.figure.Figure([6, 4.5])
    ax = fig.subplots(1, 1)
    fig.suptitle(
        r'Odvisnost ionizacijskega toka od napetosti na kondenzatorju $U_\mathrm{C}$'
    )
    
    for key, val in meas_dict.items():
        U, I_nA = val
        I = 1e-9 * I_nA
        # --- Plot ---
        ax.plot(
            U, 1e9 * I,
            alpha=0.25,
        )
        ax.scatter(
            U, 1e9 * I,
            marker='^',
            label=f'pri $U_\\mathrm{{R}} = {str.split(key, "-")[-1]} \,\mathrm{{kV}}$'
        )
    
    # --- Set ---
    ax.set(
        xlabel=r'$U_\mathrm{C}\,[\mathrm{V}]$',
        ylabel=r'$I_\mathrm{C}\,[\mathrm{nA}]$',
        xlim=(0,300),
        ylim=(0,3.8),
    )
    ax.grid(which='both', ls='--')
    fig.subplots_adjust(top=0.8)
    fig.legend(
        loc='upper center', bbox_to_anchor=(0.5, 0.94),
        ncols=2, frameon=False
    )
    
    # --- Figure I/O ---
    fig_io(fig, savefig, viewfig)
        
fig_ionization(savefig='I-by-U.pdf', viewfig=True)
'''
Izmerjene vrednosti toka $I_\mathrm{C}$ med ploščama kondenzatorja za različne napetosti na
kondenzatorju. Odvisnost je izmerjena za 4 različne napetosti na rentgenski cevi; tok na rentgenski
cevi pa je konstanten, sicer $1\,\mathrm{mA}$.
'''

'\nIzmerjene vrednosti toka $I_\\mathrm{C}$ med ploščama kondenzatorja za različne napetosti na\nkondenzatorju. Odvisnost je izmerjena za 4 različne napetosti na rentgenski cevi; tok na rentgenski\ncevi pa je konstanten, sicer $1\\,\\mathrm{mA}$.\n'

In [149]:
from matplotlib.ticker import AutoMinorLocator

def fig_ionization_saturated(
    savefig=None,
    viewfig=False,
):
    fig = mpl.figure.Figure([1.6, 4.5])
    ax = fig.subplots(1, 1)
    fig.suptitle(
        'Za razbiranje\n'
        r'zasičenega $I_\mathrm{C}$'
    )
    
    for key, val in meas_dict.items():
        U, I_nA = val
        I = 1e-9 * I_nA
        # --- Plot ---
        ax.plot(
            U, 1e9 * I,
            alpha=0.25,
        )
        ax.scatter(
            U, 1e9 * I,
            marker='.', zorder=10,
            label=f'pri $U_\\mathrm{{R}} = {str.split(key, "-")[-1]} \,\mathrm{{kV}}$'
        )
    
    # --- Set ---
    ax.set(
        xlabel=r'$U_\mathrm{C}\,[\mathrm{V}]$',
        ylabel=r'$I_\mathrm{C}\,[\mathrm{nA}]$',
        xlim=(100,300),
        ylim=(0,3.8),
        xticks=[100,300],
    )
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.grid(which='major', axis='y', ls='-')
    ax.grid(which='minor', axis='y', ls=':')
    fig.subplots_adjust(left=0.38, right=0.82)
    
    # --- Figure I/O ---
    fig_io(fig, savefig, viewfig)
        
fig_ionization_saturated(savefig='I-by-U-saturated.pdf', viewfig=True)